# latent factor model

## Basic Idea: 
$R \approx P \times Q^T = \hat{R}$ <br>
$K$ is the number of latent factors.  <br>
Adding bias in the model, we have $\hat{r_{ij}} = b+bu_i+bd_j+\sum_{k=1}^{K}p_{ik}q_{kj}$ <br>
$S$ is the set of observed user-item pair <br>
Our goal is to minimize $\sum_{(u_i,d_j,r_{ij})\in S} (r_{ij}-\sum_{k=1}^K p_{ik}q_{kj})+\frac{\beta}{2}(\sum_{k=1}^K(\|P\|^2+\|Q\|^2)$ <br>
The optimization is achieved by stochastic gradient descent algorithm <br>
encoding: purchased:5, add_to_cart: 1, viewed: 2, missing: 0 <br>
Metric: Hit rate <br>
Number of recommended items: 50 <br>

In [1]:
import numpy as np
import pandas as pd
import heapq
import time
import matplotlib.pyplot as plt

class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [2]:
class BaseLine():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Computer prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j]
        return prediction

    def full_matrix(self):
        """
        Computer the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [3]:
def train_test_split(df,proportion):
    """
    This function aims to create cross validation and training set
    Proportion controls sample size in later comparison
    """

    user=[]
    category=[]
    action=[]
    random=[]
    for person in df.index:
        for item in df.columns:
            if df.loc[person,item]>0:
                user.append(person)
                category.append(item)
                action.append(df.loc[person,item])
                random.append(np.random.rand())
    pair = pd.DataFrame({'user':user,'category':category,'action':action,'random_number':random})
    test = pair[pair['random_number']>=proportion]
    train = pair[pair['random_number']<proportion]
    keep = []
    for row in test.itertuples():
        if row[4] in list(train['user']):
            keep.append(row[0])
    test = test.loc[keep]
    train=df
    for row in test.itertuples():
        train.loc[row[4],row[2]]=0
    return train,test    

In [4]:
def Cross_Validation(train):
    """
        Arguments
        - train (dataframe) : training set
    
    """
    train.replace(['purchased','add_to_cart','viewed'],[5,2,1],inplace=True)
    train.fillna(0,inplace=True)
    

    """
    tune K, alpha, beta, iterations, number_of_recommendation on set cv
    """
    training_sub,cv = train_test_split(train,0.8)
    cv = cv.set_index(['user','category'])
    R=training_sub.values
    
    best_rate = 0
    best_model = {'K':0,'beta':0,'iterations':0}
    log = []
    n_recommended=50
    alpha = 0.01
    for K in [2,20,50,100,150,200,300]:
        for beta in [0.01,0.05,0.1]:
            for iterations in [20,50,100,200]:
                start = time.time()
                print("K:{},beta:{},iterations:{}".format(K,beta,iterations))
                mf = MF(R, K=K, alpha=alpha, beta=beta, iterations=iterations)
                mf.train()
                pred_svd = pd.DataFrame(mf.full_matrix(),index=train.index,columns=train.columns)
                rate_svd = 0
                precision = 0
                for user in pred_svd.index:
                    #get predictions for all missing cells in training matrix
                    heap_svd=[]
                    available = 0 #intercection of prediction & test
                    hit = 0 #correctly predict purchase
                    for item in pred_svd.columns:
                        if training_sub.loc[user,item] <1: 
                            heapq.heappush(heap_svd,(pred_svd.loc[user,item],item))
                    #make recommendation of the first n_recommended items
                    for (rate,item) in heapq.nlargest(n_recommended,heap_svd):
                        if (user,str(item)) in cv.index:
                            available += 1
                            if cv.loc[(user,str(item)),'action'] == 5:
                                hit += 1
                    if available > 0:
                        rate_svd += hit/available
                        precision += hit/50
                print("rate_svd is {}".format(rate_svd))
                print("precision is {}".format(precision))
                if rate_svd > best_rate:
                    best_rate = rate_svd
                    best_model['K']=K
                    best_model['beta']=beta
                    best_model['iterations']=iterations
                    
                end= time.time()
                total = end-start
                print(total)
                log.append((K,beta,iterations,rate_svd,precision,total))
    return log,training_sub,cv,best_model
    
    
    

In [5]:
def predict(best_model,training_sub,test):
    """
    use the best model from cv to do prediction
    """
    rate_svd = 0
    rate_base = 0
    alpha =0.01
    n_recommended=50
    R = training_sub.values
    test = test.set_index(['user','category'])
    if best_model['K']>0:                    
        mf = MF(R, K=best_model['K'], alpha=alpha, beta=best_model['beta'], iterations=best_model['iterations'])
        mf.train()    
        base = BaseLine(R, K=best_model['K'], alpha=alpha, beta=best_model['beta'], iterations=best_model['iterations'])
        base.train()
        pred_svd = pd.DataFrame(mf.full_matrix(),index=training_sub.index,columns=training_sub.columns)
        pred_base = pd.DataFrame(base.full_matrix(),index=training_sub.index,columns=training_sub.columns)

        for user in pred_svd.index:
            #get predictions for all missing cells in training matrix
            heap_svd=[]
            heap_base=[]
            available_svd = 0
            hit_svd = 0
            available_base = 0
            hit_base = 0
            for item in pred_svd.columns:
                if training_sub.loc[user,item] <1: 
                    heapq.heappush(heap_svd,(pred_svd.loc[user,item],item))
                    heapq.heappush(heap_base,(pred_base.loc[user,item],item))
            #make recommendation of the first n_recommended items
            for (rate,item) in heapq.nlargest(n_recommended,heap_svd):
                if (user,int(item)) in test.index:
                    available_svd += 1
                    if test.loc[(user,int(item)),'action'] == 'purchased':
                        hit_svd += 1
            if available_svd > 0:
                rate_svd +=  hit_svd/available_svd

            #using baseline model to do recommendation
            for (rate,item) in heapq.nlargest(n_recommended,heap_base):
                if (user,int(item)) in test.index:
                    available_base += 1
                    if test.loc[(user,int(item)),'action'] == 'purchased':
                        hit_base += 1
            if available_base > 0:
                rate_base +=  hit_svd/available_base
    return rate_base,rate_svd

In [6]:
df=pd.read_csv("/Users/nihaozheng/Desktop/Personalization/project/retailrocket-recommender-system-dataset/rating matrix.csv",
                   index_col='user')
train = pd.read_csv("/Users/nihaozheng/Desktop/Personalization/project/retailrocket-recommender-system-dataset/training.csv",
                   index_col='user')
test = pd.read_csv("/Users/nihaozheng/Desktop/Personalization/project/retailrocket-recommender-system-dataset/testing.csv")



/Users/nihaozheng/anaconda/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (113,128,142,164,186,191,198,213,219,238,252,272,313,331,336,351,356,358,368,379,393,396,441,445,448,449,463,468,470,471,472,480,484,492,495,511,514,524,526,534,539,540,542,544,547,552,556,559,560,563,570,574,575,579,584,587,592,595,600,602,606,609,627,628,629,630,632,635,636,638,640,644,646,651,652,653,671,675,679,680,682,685,687,688,692,695,698,703,710,714,715,717,718,720,721,723,724,725,726,730,736,740,742,744,745,747,750,751,752,753,755,757,759,760,767,771,773,778,780,781,782,784,785,788,790,791,796,805,818,820,823,824,825,826,828,829,834,835,836,837,838,839,842,843,849,850,851,852,853,854,858,861,863,864,865,867,869,871,874,875,876,878,879,888,889,890,892,907,908,912,913,914,915,916,917,918,921,922,924,926,927,928,929,930,931,932,934,935,936,938,939,940,941,943,944,950,951,952,954,956,962,963,966,970,972,974,975,976,978,979,981,982,983,984,985,986,987,996,1001,1002,1

# Cross Validation
# WARNING: DO NOT RUN THIS LINE, IT TAKES FIVE HORS FROM YOUR LIFE

In [ ]:
log,training_sub,cv,best_model = Cross_Validation(train)

## Prediction

In [7]:
#result of cross validation
best_model = {'K':100, 'beta':0.01, 'iterations':20}

In [11]:
#training data used in cv
training_sub = pd.read_csv("/Users/nihaozheng/Desktop/Personalization/project/retailrocket-recommender-system-dataset/cv_training.csv",
                          index_col='user')

In [12]:
#prediction
rate_based, rate_svd= predict(best_model,training_sub,test)

Iteration: 10 ; error = 190.6098
Iteration: 20 ; error = 127.5772
Iteration: 10 ; error = 189.9657
Iteration: 20 ; error = 10575.8709
